# Solving Poisson's Equation on an L-shaped Domain using PINNs with DeepXDE

This notebook demonstrates how to use Physics-Informed Neural Networks (PINNs) implemented with the DeepXDE library to solve Poisson's equation on a non-trivial L-shaped domain.

**Poisson's Equation:** Δu(x, y) = f(x, y)\nIn this example, we will solve for f(x,y) = 1.

**L-shaped Domain:** The domain is defined as `([0,1]x[1,1]) \\ ([0.5,0.5]x[1,1])`.

**Boundary Conditions:** We will apply Dirichlet boundary conditions, u(x, y) = 0, on all boundaries of the domain.

## 1. Setup: Import Libraries

In [ ]:
!pip install deepxde

import torch
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt

import os
os.environ['DDE_BACKEND'] = 'pytorch'

import torch
if torch.cuda.is_available():
    print("GPU is available!")
    print("Device Name:", torch.cuda.get_device_name(0))
else:
    print("GPU is not available.")

## 2. Geometry Definition
The L-shaped domain is defined by the following vertices:
P1=(0,0), P2=(1,0), P3=(1,0.5), P4=(0.5,0.5), P5=(0.5,1), P6=(0,1).

In [ ]:
vertices = [[0,0], [1,0], [1,0.5], [0.5,0.5], [0.5,1], [0,1]]
geom = dde.geometry.Polygon(vertices)

# Optional: Plot geometry to verify
plt.figure(figsize=(6,6))
plt.plot(*zip(*vertices + [vertices[0]]), 'b-', label='Domain Boundary') # Close the polygon
plt.fill(*zip(*vertices), 'lightblue', alpha=0.5) # Fill the polygon
plt.xlabel('x')
plt.ylabel('y')
plt.title('L-shaped Domain Geometry')
plt.gca().set_aspect('equal', adjustable='box')
plt.legend()
plt.grid(True)
plt.show()

## 3. PDE and Boundary Condition Definition\n\n**PDE:** Δu - 1 = 0 (Laplacian of u is equal to 1)\n\n**Boundary Condition:** u = 0 on all boundaries.

In [ ]:
def pde(x, y):\n    """Defines the Poisson equation: dy_xx + dy_yy - 1 = 0"""\n    dy_xx = dde.grad.hessian(y, x, i=0, j=0)\n    dy_yy = dde.grad.hessian(y, x, i=1, j=1)\n    return dy_xx + dy_yy - 1.0\n\ndef boundary_func(_, on_boundary):\n    return on_boundary\n\ndef func_boundary_val(x):\n    return np.zeros((x.shape[0], 1), dtype=dde.config.real(np))\n\nbc = dde.DirichletBC(\n    geom,\n    func_boundary_val, \n    boundary_func\n)

## 4. Problem Setup (Data)

In [ ]:
N_domain = 2000\nN_boundary = 500\nN_test = 500\ndata = dde.data.PDE(\n    geom,\n    pde,\n    bc,\n    num_domain=N_domain,\n    num_boundary=N_boundary,\n    num_test=N_test,\n    solution=None\n)

## 5. Network and Model Creation

In [ ]:
input_dim = 2\noutput_dim = 1\nnum_hidden_layers = 4\nneurons_per_layer = 50\nactivation_fn = "tanh"\nkernel_initializer = "Glorot uniform"\nnet = dde.maps.FNN(\n    [input_dim] + [neurons_per_layer] * num_hidden_layers + [output_dim],\n    activation_fn,\n    kernel_initializer\n)\nmodel = dde.Model(data, net)

## 6. Model Training

In [ ]:
learning_rate = 1e-3\nmodel.compile("adam", lr=learning_rate)\nnum_iterations = 20000\nprint(f"Starting training for {num_iterations} iterations...")\nlosshistory, train_state = model.train(iterations=num_iterations)\nprint("Training finished.")

## 7. Results and Visualization

In [ ]:
dde.utils.plot_loss_history(losshistory)\nplt.title('Loss History (Log Scale)')\nplt.show()\nplot_resolution = 100\nx_plot = np.linspace(0, 1, plot_resolution)\ny_plot = np.linspace(0, 1, plot_resolution)\nX_plot, Y_plot = np.meshgrid(x_plot, y_plot)\nplot_points_flat = np.vstack((X_plot.ravel(), Y_plot.ravel())).T\nis_inside_domain = geom.inside(plot_points_flat)\nplot_points_inside = plot_points_flat[is_inside_domain]\nif plot_points_inside.shape[0] > 0:\n    u_pred_inside = model.predict(plot_points_inside)\n    plt.figure(figsize=(8, 6))\n    plt.tricontourf(plot_points_inside[:, 0], plot_points_inside[:, 1], u_pred_inside.ravel(), levels=100, cmap='viridis')\n    plt.colorbar(label='Predicted u(x,y)')\n    plt.xlabel('x')\n    plt.ylabel('y')\n    plt.title('Predicted Solution u(x,y) on L-shaped Domain')\n    plt.gca().set_aspect('equal', adjustable='box')\nelse:\n    plt.figure(figsize=(8, 6))\n    plt.text(0.5, 0.5, \"No points found inside domain for plotting\", ha='center', va='center')\n    plt.xlabel('x')\n    plt.ylabel('y')\n    plt.title('Predicted Solution u(x,y) on L-shaped Domain')\nplt.show()

## 8. Note on Analytical Solution for L-shaped Domain\n\nFor the Poisson equation Δu = 1 on the L-shaped domain `([0,1]x[0,1]) \\ ([0.5,1]x[0.5,1])` with homogeneous Dirichlet boundary conditions (u=0 on all boundaries), finding a simple, closed-form analytical solution is significantly more challenging than for simpler geometries like rectangles or circles.\n\nThe primary difficulties arise from:\n1.  **Re-entrant Corner:** The L-shape has a re-entrant corner. The solution typically exhibits a singularity at this corner, meaning its derivatives can become unbounded. Standard separation of variables techniques are difficult to apply directly.\n2.  **Inhomogeneous PDE:** The source term f=1 makes it an inhomogeneous Poisson equation.\n\nAnalytical solutions, when derived, often involve complex series expansions or specialized numerical techniques. Due to this complexity, a straightforward analytical function is not readily available for direct implementation and comparison in this notebook.\n\nFor this notebook, we will proceed to add an FEM solution as another numerical benchmark for the PINN solution.

## 9. Finite Element Method (FEM) Solution with FEniCSx for L-shaped Domain\n\nWe will now solve Poisson's equation (Δu = 1) with u=0 on the boundary of the L-shaped domain using the Finite Element Method with FEniCSx. This provides another numerical solution to compare with the PINN result.\n\n**Note:** Running the following cells requires a working installation of FEniCSx (including `dolfinx` and `gmsh` for meshing). If FEniCSx is not installed, these cells will raise an error.

In [ ]:
try:\n    import dolfinx\n    import dolfinx.fem as fem\n    import dolfinx.mesh as dmesh\n    from dolfinx.fem.petsc import LinearProblem\n    import ufl\n    from mpi4py import MPI\n    import gmsh\n    lshape_fenicsx_available = True\n    print(f\"FEniCSx version: {dolfinx.__version__} (for L-shape)\")\nexcept ImportError:\n    lshape_fenicsx_available = False\n    print(\"FEniCSx is not installed. Skipping FEM solution cells for L-shaped domain.\")\nif lshape_fenicsx_available:\n    lshape_comm = MPI.COMM_WORLD\n    lshape_model_rank = 0

In [ ]:
if lshape_fenicsx_available:\n    gdim_lshape = 2\n    gmsh.initialize()\n    if lshape_comm.rank == lshape_model_rank:\n        gmsh.model.add(\"l_shape_fem\")\n        gmsh_pts = []\n        for i, v_coord in enumerate(vertices):\n            gmsh_pts.append(gmsh.model.occ.addPoint(v_coord[0], v_coord[1], 0, tag=i+1))\n        gmsh_lines = []\n        for i in range(len(gmsh_pts)):\n            p1_tag = gmsh_pts[i]\n            p2_tag = gmsh_pts[(i + 1) % len(gmsh_pts)]\n            gmsh_lines.append(gmsh.model.occ.addLine(p1_tag, p2_tag, tag=i+1))\n        curve_loop = gmsh.model.occ.addCurveLoop(gmsh_lines, tag=1)\n        surface = gmsh.model.occ.addPlaneSurface([curve_loop], tag=1)\n        gmsh.model.occ.synchronize()\n        gmsh.model.addPhysicalGroup(1, gmsh_lines, tag=1)\n        gmsh.model.addPhysicalGroup(2, [surface], tag=2)\n        mesh_size = 0.05\n        gmsh.option.setNumber(\"Mesh.CharacteristicLengthMin\", mesh_size)\n        gmsh.option.setNumber(\"Mesh.CharacteristicLengthMax\", mesh_size)\n        gmsh.model.mesh.generate(gdim_lshape)\n    lshape_domain, lshape_cell_tags, lshape_facet_tags = dolfinx.io.gmshio.model_to_mesh(\n        gmsh.model, lshape_comm, lshape_model_rank, gdim=gdim_lshape\n    )\n    gmsh.finalize()\n    lshape_tdim = lshape_domain.topology.dim\n    lshape_fdim = lshape_tdim - 1

In [ ]:
if lshape_fenicsx_available:\n    V_lshape = fem.FunctionSpace(lshape_domain, ("Lagrange", 1))\n    all_boundary_facets = lshape_facet_tags.find(1)\n    u_D_lshape = fem.Constant(lshape_domain, dolfinx.default_scalar_type(0.0))\n    bc_lshape_fem = fem.dirichletbc(u_D_lshape, fem.locate_dofs_topological(V_lshape, lshape_fdim, all_boundary_facets), V_lshape)\n    bcs_lshape_fem = [bc_lshape_fem]

In [ ]:
if lshape_fenicsx_available:\n    u_lshape = ufl.TrialFunction(V_lshape)\n    v_lshape = ufl.TestFunction(V_lshape)\n    f_source_lshape = fem.Constant(lshape_domain, dolfinx.default_scalar_type(-1.0))\n    a_lshape = ufl.dot(ufl.grad(u_lshape), ufl.grad(v_lshape)) * ufl.dx\n    L_lshape = f_source_lshape * v_lshape * ufl.dx\n    problem_lshape = LinearProblem(a_lshape, L_lshape, bcs=bcs_lshape_fem, \n                                   petsc_options={\"ksp_type\": \"preonly\", \"pc_type\": \"lu\"})\n    uh_fem_lshape = problem_lshape.solve()\n    uh_fem_lshape.name = \"u_FEM_Lshape\"

In [ ]:
if lshape_fenicsx_available:\n    if 'plot_points_inside' in locals() and plot_points_inside.shape[0] > 0:\n        points_for_eval_lshape = np.zeros((3, plot_points_inside.shape[0]))\n        points_for_eval_lshape[:gdim_lshape, :] = plot_points_inside.T\n        bb_tree_lshape = dolfinx.geometry.BoundingBoxTree(lshape_domain, lshape_domain.topology.dim)\n        cell_candidates_lshape = dolfinx.geometry.compute_collisions_points(bb_tree_lshape, points_for_eval_lshape)\n        colliding_cells_lshape = dolfinx.geometry.compute_colliding_cells(lshape_domain, cell_candidates_lshape, points_for_eval_lshape)\n        u_fem_lshape_on_grid = np.full(plot_points_inside.shape[0], np.nan)\n        points_on_proc_lshape = []\n        cells_fem_lshape_plot = []\n        original_indices_lshape_plot = []\n        for i, point in enumerate(points_for_eval_lshape.T):\n            if len(colliding_cells_lshape.links(i)) > 0:\n                points_on_proc_lshape.append(point)\n                cells_fem_lshape_plot.append(colliding_cells_lshape.links(i)[0])\n                original_indices_lshape_plot.append(i)\n        if points_on_proc_lshape:\n            u_fem_values_at_points_lshape = uh_fem_lshape.eval(np.array(points_on_proc_lshape).T, cells_fem_lshape_plot)\n            for idx_map, val in zip(original_indices_lshape_plot, u_fem_values_at_points_lshape):\n                 u_fem_lshape_on_grid[idx_map] = val\n            valid_lshape_fem_plot_indices = ~np.isnan(u_fem_lshape_on_grid)\n            plt.figure(figsize=(8, 6))\n            contour_fem_lshape = plt.tricontourf(\n                plot_points_inside[valid_lshape_fem_plot_indices, 0],\n                plot_points_inside[valid_lshape_fem_plot_indices, 1],\n                u_fem_lshape_on_grid[valid_lshape_fem_plot_indices].ravel(),\n                levels=100,\n                cmap='viridis'\n            )\n            plt.colorbar(contour_fem_lshape, label='FEM u(x,y) for L-shape')\n            plt.xlabel('x')\n            plt.ylabel('y')\n            plt.title('FEM Solution on L-shaped Domain (on evaluation points)')\n            plt.gca().set_aspect('equal', adjustable='box')\n            plt.show()\n        else:\n            print(\"L-shape FEM: No evaluation points found on the current MPI rank or in cells for plotting.\")\n    else:\n        print(\"L-shape FEM Plot: PINN plotting points `plot_points_inside` not available or empty.\")

## 10. Conclusion\n\nThis notebook demonstrated the setup and solution of Poisson's equation (Δu = 1) on an L-shaped domain using DeepXDE (a PINN approach).\n\nKey steps for the PINN solution included:\n1. Defining the L-shaped geometry.\n2. Setting up the Poisson PDE (Δu=1) and homogeneous Dirichlet boundary conditions (u=0).\n3. Configuring the PINN model.\n4. Training the model and visualizing its solution and loss history.\n\nThe notebook was then extended to include:\n1. **Discussion on Analytical Solution:** An explanation of why obtaining a simple analytical solution for this problem is challenging due to the geometry (re-entrant corner) and PDE form.\n2. **FEM Solution:** An example implementation using FEniCSx to solve the same Poisson problem, providing a numerical benchmark. This included mesh generation for the L-shape, boundary condition application, variational formulation, solving, and visualization.\n\nComparing the PINN solution with the FEM solution provides a way to assess the PINN's accuracy in the absence of a simple analytical solution.\n\nFurther explorations could involve:\n- More detailed comparison between PINN and FEM, possibly calculating error norms if a common high-resolution grid is used.\n- Investigating mesh refinement strategies for FEM, especially near the re-entrant corner, to capture singularities more accurately.\n- Applying adaptive refinement or weighting techniques in PINNs to better handle solution complexities near the corner.